## Data Preprocessing

In [3]:
import pandas as pd
import numpy as np

filepath = r'..\Database\TBIMSPublic.2024-11-01\Data\Form1.csv'

df = pd.read_csv(filepath)

C:\Users\benol\AppData\Local\Temp\ipykernel_35040\542096908.py:6: DtypeWarning: Columns (17,18,19,291) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [ ]:
df.info()
#print(df.notnull().sum())
#df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19560 entries, 0 to 19559
Columns: 371 entries, Mod1Id to YoungestAgeTBI
dtypes: float64(317), int64(50), object(4)
memory usage: 55.4+ MB


Hypertension
66    19560
Name: count, dtype: int64

In [31]:
# import missingno as msno

# msno.matrix(df)

mixed_type_col = [17,18,19,291]

mixed_type_col_names =[df.columns[column] for column in mixed_type_col]

print(mixed_type_col_names)

# for col in mixed_type_col_names:
#     print(df[col].dtypes)

df['FIMLocoModeD'].value_counts()

['DeathCause1', 'DeathCause2', 'DeathECode', 'FIMLocoModeD']


FIMLocoModeD
w     14323
c      3432
66      273
99      178
Name: count, dtype: int64

In [26]:
form_vars_filepath = r'..\Database\TBIMSPublic.2024-11-01\Code\Form1_Variables.csv'
form_vars_df = pd.read_csv(form_vars_filepath)

form_vars_df[form_vars_df['Name'].isin(mixed_type_col_names)]

,VariableId,Name,Question,MaxRealValueLength
17,537.0,DeathCause1,Primary cause of death ICD diagnosis code:,6.0
18,538.0,DeathCause2,Secondary cause of death ICD diagnosis code:,4.0
19,539.0,DeathECode,External cause of death ICD code:,4.0
291,671.0,FIMLocoModeD,Walking/Wheelchair mode:,1.0


In [12]:
form_vars_df[form_vars_df['Name'].isin(mixed_type_col_names)]

,VariableId,Name,Question,MaxRealValueLength
17,537.0,DeathCause1,Primary cause of death ICD diagnosis code:,6.0
18,538.0,DeathCause2,Secondary cause of death ICD diagnosis code:,4.0
19,539.0,DeathECode,External cause of death ICD code:,4.0
291,671.0,FIMLocoModeD,Walking/Wheelchair mode:,1.0


In [ ]:
import re

def extract_labels(sps_file_path):
    with open(sps_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # ---- Extract VARIABLE LABELS ----
    var_label_dict = {}
    var_label_blocks = re.findall(r"VARIABLE LABELS\s+(.*?)(?=\.\s*(?:VALUE LABELS|EXECUTE|$))", content, re.DOTALL)
    for block in var_label_blocks:
        matches = re.findall(r"(\w+)\s+'([^']+)'", block)
        for var, label in matches:
            var_label_dict[var] = label

    # ---- Extract VALUE LABELS ----
    value_label_dict = {}
    value_blocks = re.findall(r"VALUE LABELS\s+(.*?)(?=\.\s*(?:VARIABLE LABELS|EXECUTE|$))", content, re.DOTALL)
    for block in value_blocks:
        # Supports multi-variable assignments like: VALUE LABELS var1 var2 1 "Yes" 2 "No".
        parts = re.split(r'\s+(?=\d+\s+\')', block.strip())  # split before value-label pairs
        header = parts[0].strip().split()
        body = " ".join(parts[1:])

        # Extract value-label pairs
        label_pairs = re.findall(r"(\d+)\s+'([^']+)'", body)
        mapping = {int(k): v for k, v in label_pairs}

        # Assign the same mapping to each variable listed
        for var in header:
            value_label_dict[var] = mapping

    return var_label_dict, value_label_dict

var_labels_filepath =  r'..\Database\TBIMSPublic.2024-11-01\Code\Form1.sps'


var_labels, value_labels = extract_labels(var_labels_filepath)

for col in mixed_type_col_names:
    print(value_labels[col])

{44444: 'Person Expired But Cause Unknown', 88888: 'Not Applicable', 99999: 'Unknown if Person Expired'}
{44444: 'Person Expired But Cause Unknown', 88888: 'Not Applicable', 99999: 'Unknown if Person Expired'}
{44444: 'Person Expired But Cause Unknown', 88888: 'Not Applicable', 99999: 'Unknown if Person Expired'}
{66: 'Variable did not exist', 99: 'Unknown: Assessed at more than 72 hours'}
